In [64]:
import numpy as np
import gym
import os
from progressbar import ProgressBar
from time import sleep
import scipy
from matplotlib import image

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from keras import backend as K

env = gym.make('Breakout-v0')
state_size = env.observation_space.shape[0]
action_size = env.env.action_space.n

In [4]:
# create the networks
# input 84x84x4
# convo "32 filters 8x8 stride 4", relu
# convo "64 filters 4x4 stride 2", relu
# convo "64 filters 4x4 stride 2", relu
# convo "64 filters 3x3 stride 1", relu
# 512 fully connected rectifier
# output (action-size) fully connected, linear

# make a hat, and a q hat

In [72]:
def pre_process_frame(frame, prev_frame):
    r = frame[:,:,0] # red channel
    g = frame[:,:,1] # blue channel
    b = frame[:,:,2] # green channel
    l = (0.2126*r + 0.7152*g + 0.0722*b) # luminescence 
    max_r = max(np.max(prev_frame[:,:,0]), np.max(r))
    max_g = max(np.max(prev_frame[:,:,1]), np.max(g))
    max_b = max(np.max(prev_frame[:,:,2]), np.max(b))
    
    
    normalized_color = np.zeros((frame.shape[0], frame.shape[1], 3))
    normalized_color[:,:,0] = r.astype(np.float64) / max_r
    normalized_color[:,:,1] = g.astype(np.float64) / max_g
    normalized_color[:,:,2] = b.astype(np.float64) / max_b
#     normalized_color[:,:,3] = l
    
    
    phi_frame = np.zeros((84, 84, 3))
    zoom_shape = np.array(phi_frame.shape, dtype=float) / np.array(normalized_color.shape, dtype=float)
    phi_frame = scipy.ndimage.zoom(normalized_color, zoom_shape)
    return phi_frame

state = env.reset()
done = False
lives = -1
prev_frame = np.zeros_like(state) # the "prevvious" frame is nothing
i = 0

while not done:
    i += 1
    old_lives = lives
#     env.render()
    action = np.random.randint(action_size)
    frame, reward, done, ale_lives = env.step(action)
    lives = ale_lives['ale.lives']
    phi_frame = pre_process_frame(frame, prev_frame)  
    image.imsave(
        "/home/rharriso/Desktop/frames/frame-%s.png" % i,
        (phi_frame * 200).astype(np.uint8)
    )
    
#     if lives != old_lives or not np.isclose(reward, 0.0):
#         print(reward, done, lives)
    
    lives = ale_lives['ale.lives']
    
    # preprocess frame
    # max of each color channel last two frames (flicker)
    # extract a Y channel (luminecense)
    
    
    prev_frame = frame